In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]
test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [5]:
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]=len(word_to_ix)
print(word_to_ix)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [28]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        # Define the parameters that you will need.
        self.linear = nn.Linear(vocab_size, num_labels)
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec),dim = 1)

def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1,-1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

for params in model.parameters():
    print(params)

sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Parameter containing:
tensor([[ 0.0284, -0.1134,  0.0733, -0.0134, -0.0508,  0.1052,  0.1402,  0.1908,
          0.0888, -0.0054,  0.1603,  0.1561, -0.0983,  0.0893,  0.0230, -0.0848,
         -0.0467, -0.0463, -0.1372, -0.0598,  0.0660,  0.1440,  0.0149, -0.1899,
          0.1039, -0.0593],
        [ 0.1751, -0.1895, -0.0459,  0.1521,  0.0848, -0.0978, -0.1698, -0.0021,
          0.0758, -0.1628, -0.1641, -0.0025, -0.0402,  0.1261, -0.1900, -0.0312,
         -0.1520, -0.0286, -0.1002,  0.0617, -0.1641, -0.1543, -0.1113, -0.1718,
          0.1238, -0.0281]], requires_grad=True)
Parameter containing:
tensor([-0.1090,  0.1374], requires_grad=True)
tensor([[-0.8520, -0.5561]], grad_fn=<LogSoftmaxBackward>)


In [21]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [32]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    lob_probs = model(bow_vec)
    print(log_probs)

print(next(model.parameters())[:, word_to_ix["creo"]])
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

tensor([[-0.8520, -0.5561]], grad_fn=<LogSoftmaxBackward>)
tensor([[-0.8520, -0.5561]], grad_fn=<LogSoftmaxBackward>)
tensor([ 0.1603, -0.1641], grad_fn=<SelectBackward>)


In [33]:
for epoch in range(100):
    for instance, label in data:
        model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
        
        log_probs = model(bow_vec)
        
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([[-0.1302, -2.1033]], grad_fn=<LogSoftmaxBackward>)
tensor([[-2.6291, -0.0749]], grad_fn=<LogSoftmaxBackward>)
tensor([ 0.5693, -0.5731], grad_fn=<SelectBackward>)
